In [1]:
import pandas as pd

In [2]:
def make_arbStep(estep,dwell,filename,scale = pd.Series(1.2**(np.linspace(-4,4,9))),mode = 'HiTH'):
    volt_step = {'HiTH':pd.DataFrame({'U+':[61.91,119.41,229.42,451.48,870.61,1782.40,3452],
                            'U-':[30.55,58.92,113.19,222.75,429.54,879.40,1701]}),
                 'HiRes':pd.DataFrame({'U+':[40.30,77.73,149.34,293.89,566.72,1160.24,2248.26],
                            'U-':[13.65,26.33,50.59,99.56,191.98,393.03,761.60]})}
    
    things = volt_step[mode].loc[estep-1].apply(lambda x: x*scale)
    
    maxN = 1702
    maxP = 3454
    locr = (things.T['U-']<maxN)&(things.T['U+']<maxP)
    things = things.T.loc[locr].T

    fup = lambda volt: 'send ILO_ESA_P_RMP(TARGET = %.2f,STEPS = 3447,DWELL = %d)'%(volt,0)
    fdn =lambda volt: 'send ILO_ESA_N_RMP(TARGET = %.2f,STEPS = 1700,DWELL = %d)'%(volt,0)

    cmds = things.apply(lambda x :'\n'.join([fup(x['U+']),fdn(x['U-'])]))
    
    with open(filename,'w') as f: 
        f.write('script %s() \nbegin'%filename.strip('.gstol'))
        cmds.apply(lambda x: f.write('\n%s \nwait timeout %d\n'%(x,dwell)))
        f.write('end')
        f.close()

In [3]:
estep_norm =pd.Series([0.0347,0.0670,0.1287,0.2533,0.4884,1.0000,1.935])

In [15]:
make_arbStep(6,1000,'DER2_HiTh_Esteps.gstol',estep_norm,'HiTH')
make_arbStep(6,1000,'DER2_HiRes_Esteps.gstol',estep_norm,'HiRes')

In [16]:
for i in range(1,8):
    stuff = make_arbStep(i,1000,'DER1_HiTh_estep%d.gstol'%i,scale = pd.Series(1.2**(np.linspace(-4,4,9))),mode = 'HiTH')
    stuff = make_arbStep(i,1000,'DER1_HiRes_estep%d.gstol'%i,scale = pd.Series(1.1**(np.linspace(-4,4,9))),mode = 'HiRes')